## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-14-54-41 +0000,bbc,UK growth forecasts lowered from next year,https://www.bbc.com/news/articles/cg5m7mg8p59o...
1,2025-11-26-14-53-38 +0000,bbc,At least 13 dead as fire engulfs Hong Kong tow...,https://www.bbc.com/news/articles/cvg165d7ppvo...
2,2025-11-26-14-52-20 +0000,nyt,Donors to Trump’s Transition Revealed a Year L...,https://www.nytimes.com/2025/11/26/us/politics...
3,2025-11-26-14-47-07 +0000,nypost,Singer Tish Hyman slams Eric Swalwell for dodg...,https://nypost.com/2025/11/26/us-news/singer-t...
4,2025-11-26-14-44-54 +0000,nyt,Live Updates: At Least 13 Are Killed as Fire E...,https://www.nytimes.com/live/2025/11/26/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,47
5,year,20
66,ukraine,18
58,new,18
76,peace,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...,124
64,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,122
288,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,119
168,2025-11-26-02-22-37 +0000,wapo,"Trump pushes Ukraine, Russia on peace deal, bu...",https://www.washingtonpost.com/world/2025/11/2...,111
146,2025-11-26-07-17-02 +0000,bbc,Trump's envoy to have Ukraine peace plan talks...,https://www.bbc.com/news/articles/cy95jvw57v2o...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,124,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...
64,75,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
74,52,2025-11-26-11-10-35 +0000,nypost,FDA chief promises ‘powerful’ new treatments f...,https://nypost.com/2025/11/26/us-news/fda-chie...
263,52,2025-11-25-21-14-00 +0000,wsj,At the White House’s annual—and typically stai...,https://www.wsj.com/politics/policy/trump-pard...
106,46,2025-11-26-10-15-47 +0000,nypost,Hong Kong high-rise fire kills 14 as blaze rip...,https://nypost.com/2025/11/26/world-news/hong-...
79,42,2025-11-26-11-00-00 +0000,latimes,Chabria: California's first partner pushes to ...,https://www.latimes.com/california/story/2025-...
14,42,2025-11-26-14-21-01 +0000,nypost,Trump lashes out at NY Times reporter in lates...,https://nypost.com/2025/11/26/us-news/trumps-o...
340,42,2025-11-25-16-09-36 +0000,nypost,Ukraine peace plan expected to be rejected by ...,https://nypost.com/2025/11/25/world-news/ukrai...
134,34,2025-11-26-08-58-31 +0000,wapo,Taiwan plans to spend $40 billion on U.S. arms...,https://www.washingtonpost.com/world/2025/11/2...
53,32,2025-11-26-12-10-59 +0000,wapo,Two hostages’ bodies remain in Gaza as another...,https://www.washingtonpost.com/world/2025/10/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
